Este código é referente ao blog:

Título: [PT] Papai Noel Encontra a IA Generativa: Decifrando Cartas de Natal Escritas à Mão com LLM, LangChain e Elasticsearch

Author: Alex Salgado

Link: https://discuss.elastic.co/t/dec-22nd-2023-en-santa-claus-meets-genai-deciphering-handwritten-christmas-letters-with-llm-langchain-and-elasticsearch/347311


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/salgado/music-search/blob/main/elastic_music_search.ipynb)


# Santa Claus Meets AI: Deciphering Handwritten Christmas Letters with LLM, LangChain and Elasticsearch

No coração do Polo Norte, a equipe de duendes de Papai Noel enfrentava um desafio logístico formidável: como lidar com milhões de cartas de crianças de todo o mundo. Com um olhar determinado, Papai Noel decidiu que era hora de incorporar inteligência artificial na operação natalina.

Sentando-se em seu computador, equipado com o mais recente em tecnologia de IA, Papai Noel começou a trabalhar em um script Python no Jupyter Notebook. O objetivo era simples, mas ambicioso: utilizar Large Language Models (LLM) e LangChain para interpretar cartas manuscritas e extrair os dados necessários, inserindo-os de forma organizada no Elasticsearch.


In [23]:
!pip install python-dotenv elasticsearch langchain openai


O primeiro passo foi configurar as variaveis de ambiente as quais seriam utilizadas como credenciais para acesso a api do openAi e do Elasticsearch.

In [24]:
import os
from dotenv import load_dotenv

# Substitua 'path/to/your/.env' pelo caminho correto até o seu arquivo .env no Google Drive
env_path = '/content/drive/MyDrive/@Blogs/04-Advent-2023/env_advent'
load_dotenv(env_path)

# OpenAI API Key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"

# Elastic cloud credentials
es_cloud_id = os.getenv('cloud_id')
es_user = os.getenv('cloud_user')
es_pass = os.getenv('cloud_pass')


In [25]:
from PIL import Image
import requests
import numpy as np


A seguir, com uma imagem digitalizada de uma carta de Natal, Papai Noel escreveu um script para extrair o texto usando o "gpt-4-vision-preview". Essa etapa crucial transformou a escrita manual em texto digital.

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage

image_path = 'https://i.imgur.com/JNK7hww.png'

chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=512)
result = chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "Qual é a imagem? Por favor, forneça uma introdução detalhada em portugues."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_path,
                        "detail": "auto",
                    },
                },
            ]
        )
    ]
)


print(result.content)

A imagem apresenta uma carta escrita à mão, que parece ser uma carta de uma criança para o Papai Noel. O texto está em português e é possível ler o seguinte:

```
Querido Papai Noel,

Como vai você?

Meu nome é Maria, tenho 8 anos e moro no Rio de Janeiro. Estou escrevendo esta carta para pedir-lhe um presente de Natal.

Fui uma boa menina este ano. Fiz minha lição de casa, ajudei minha mãe e meu pai nos afazeres domésticos e fui gentil com meus amigos.

Para o Natal deste ano, tenho alguns desejos. Aqui está minha lista:

- Barbie Dreamhouse Adventures.
- My Little Pony.

Sei que você está muito ocupado, mas espero que possa trazer meu presente para mim.

Obrigada, Papai Noel

Com amor,

Maria
```

A carta é um exemplo típico de uma mensagem que as crianças costumam enviar ao Papai Noel antes do Natal, expressando bons comportamentos e pedindo presentes. A caligrafia parece infantil, o que é coerente com a idade da autora mencionada, Maria, que diz ter 8 anos. Os presentes que ela ped

Em seguida, o LangChain entrou em ação, analisando o texto e identificando elementos-chave como o nome da criança e a lista de desejos.

In [31]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

chain = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024)

prompt = PromptTemplate.from_template(
"""
Extract the list and child's name from the text below and return the data in JSON format using the following name:
- "child_name", "wishlist".

{santalist}

"""
)

runnable = prompt | chain | StrOutputParser()

In [32]:
letter = result.content
wishlist = runnable.invoke({"santalist": letter})
print(wishlist)

{
  "child_name": "Maria",
  "wishlist": [
    "Barbie Dreamhouse Adventures",
    "My Little Pony"
  ]
}


O Papai Noel decidiu enriquecer um pouco a base de dados, e pediu ainda, que a IA estime o peso desses presentes. Assim ele pode gerar uma lista no Kibana com os presentes das crianças divididadas em cada sacola e que couberem dentro do espaço de um trenó, que organização!!

In [33]:
chain = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024)

prompt = PromptTemplate.from_template(
"""

{santalist_json}

From the JSON above, include a new attribute in the JSON called 'weight',
which will calculate the total estimated weight of each item in the list in kilograms.
You will first need to estimate the weight of each item individually.
After that, sum these values to obtain the total weight.
Extract only the numerical value.


"""
)

runnable = prompt | chain | StrOutputParser()

In [34]:
new_wishlist = runnable.invoke({"santalist_json": wishlist})
print(new_wishlist)

{
  "child_name": "Maria",
  "wishlist": [
    "Barbie Dreamhouse Adventures",
    "My Little Pony"
  ],
  "weight": 0.5
}


In [35]:
# Insert into Elasticsearch

Agora, com os dados estruturados, era hora de movê-los para o Elasticsearch.

In [36]:
from elasticsearch import Elasticsearch

In [37]:
es = Elasticsearch(cloud_id=es_cloud_id,
                  basic_auth=(es_user, es_pass)
                  )
es.info() # should return cluster info


ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': 'c731f6bbb8314543bb3648440b501e47', 'cluster_uuid': 'pdZVQFRuTr2u3yh4l0sZyg', 'version': {'number': '8.11.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '76013fa76dcbf144c886990c6290715f5dc2ae20', 'build_date': '2023-12-05T10:03:47.729926671Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [38]:
import json

In [39]:
# Parse the JSON string
json_string = new_wishlist
data = json.loads(json_string)

# Index name
index_name = "santa_claus_list"

# Index the document
response = es.index(index=index_name, document=data)

# Print the response from Elasticsearch
print(response)

{'_index': 'santa_claus_list', '_id': 'nV8XhIwBrZtDJJrwaZX4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 9, '_primary_term': 1}


Usando o Kibana, uma interface de visualização do Elasticsearch, Papai Noel e os duendes poderiam então facilmente buscar e analisar os dados. Isso permitia uma visão clara das tendências de presentes deste ano, as localizações mais frequentes das cartas, e até mesmo identificar aquelas cartas que expressavam desejos particulares ou urgentes.

In [ ]:
 # Como nessa query usando ES|QL

POST /_query?format=txt
{
  "query": """
FROM santa_claus_list
| STATS  sum_toy = SUM(weight) BY child_name
| LIMIT 100
  """
}

# result
    sum_toy    |  child_name
---------------+---------------
30.5           |Maria
1.5            |Mike
3.0            |Theo
2.5            |Isabella
40.0           |William
30.0           |Olivia



Graças a esta solução inovadora, Papai Noel não só conseguiu atender aos pedidos de forma mais eficiente, mas também ganhou insights valiosos sobre as alegrias e esperanças das crianças ao redor do mundo, tudo graças ao poder da IA, LangChain, e Elasticsearch. Este Natal prometia ser o mais mágico e bem organizado de todos!